In [ ]:
from pandas.core.dtypes.cast import astype_dt64_to_dt64tz
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
import random
# Load the Iris dataset
iris = load_iris()

# Create a pandas DataFrame from the dataset
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])
df['target'] = df['target'].map({0:'setosa', 1:'versicolor', 2:'virginica'})

# print(df[:10])
# Split the data into training and testing sets
train_data = df.sample(frac=0.7, random_state=random.randint(0,100))
test_data = df.drop(train_data.index)

grouped = train_data.groupby('target')






In [ ]:
# Calculate the mean and standard deviation of each feature for each target class
means = grouped.mean()
stds = grouped.std()
print(means)
print(stds)

            sepal length (cm)  sepal width (cm)  petal length (cm)  \
target                                                               
setosa               4.990909          3.378788           1.448485   
versicolor           5.930769          2.756410           4.235897   
virginica            6.442424          2.900000           5.448485   

            petal width (cm)  
target                        
setosa              0.227273  
versicolor          1.307692  
virginica           1.990909  
            sepal length (cm)  sepal width (cm)  petal length (cm)  \
target                                                               
setosa               0.357390          0.377291           0.164167   
versicolor           0.541029          0.346254           0.471552   
virginica            0.607263          0.278388           0.477704   

            petal width (cm)  
target                        
setosa              0.094448  
versicolor          0.197863  
virginica          

In [ ]:
# Define a function to calculate the probability density function
def pdf(x, mean, std):
    exponent = np.exp(-(np.power(x-mean,2)/(2*np.power(std,2))))
    return (1 / (np.sqrt(2*np.pi) * std)) * exponent

# Define a function to predict the target class for a single data point
def predict(data_point):
    # Calculate the prior probability of each target class
    priors = {}
    for target, group in grouped:
        priors[target] = len(group) / len(train_data)
    
    # Calculate the likelihood of the data point given each target class
    likelihoods = {}
    for target, group in grouped:
        likelihood = 1.0
        for feature in iris['feature_names']:
            mean = means[feature][target]
            std = stds[feature][target]
            x = data_point[feature]
            likelihood *= pdf(x, mean, std)
        likelihoods[target] = likelihood
    
    # Calculate the posterior probability of each target class given the data point
    posteriors = {}
    for target in priors:
        posteriors[target] = priors[target] * likelihoods[target]
    
    # Return the target class with the highest posterior probability
    return max(posteriors, key=posteriors.get)

# Make predictions for each data point in the test set
predictions = []
for index, row in test_data.iterrows():
    predictions.append(predict(row))
print(predictions)
# Print the accuracy of the classifier
correct = 0
for i in range(len(predictions)):
    if predictions[i] == test_data.iloc[i]['target']:
        correct += 1
accuracy = correct / len(predictions)
print('Accuracy:', accuracy*100)

['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'virginica', 'versicolor', 'versicolor', 'virginica', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica', 'virginica']
Accuracy: 95.55555555555556
